In [9]:
import numpy as np 
np.set_printoptions(threshold=10000,suppress=True) 
import pandas as pd
import warnings
import matplotlib.pyplot as plt 
warnings.filterwarnings('ignore')

In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier,AdaBoostClassifier,ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import KFold,cross_val_score
import time
import xgboost as xgb
from sklearn.preprocessing import StandardScaler

clfs = {
    'RF': RandomForestClassifier(n_estimators=100, random_state=1), 
    'BAG': BaggingClassifier(base_estimator=DecisionTreeClassifier(random_state=1),n_estimators=100,random_state=1),
    'ADA': AdaBoostClassifier(n_estimators=100,random_state=1),
    'ET': ExtraTreesClassifier(n_estimators=100,random_state=1),
    'KNN': KNeighborsClassifier(n_neighbors=5),
    'CART': DecisionTreeClassifier(criterion='gini',random_state=1),
    'ID3': DecisionTreeClassifier(criterion='entropy',random_state=1),
    'Stumb': DecisionTreeClassifier(criterion='gini',max_depth=1,random_state=1),
    'MLP': MLPClassifier(hidden_layer_sizes=(20,10),random_state=1)


}

def run_classifieurs(X,Y,clfs):
    kf = KFold(n_splits=10, shuffle=True, random_state=1) 
    for i in clfs:
        clf = clfs[i]
        debut=time.time()
        cv_acc = cross_val_score(clf, X, Y, cv=kf,scoring='precision')
        fin=time.time()
        print("Accuracy for {0} is: {1:.3f} +/- {2:.3f} en {3:.3f}s".format(i,
                                                                            np.mean(cv_acc),
                                                                            np.std(cv_acc),fin-debut))

C:\Users\JeanPhilippeNdri\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
import pandas as pd
df = pd.read_csv(r'C:\Users\JeanPhilippeNdri\Desktop\Fortuneo\data_train.csv', sep=',', decimal='.')
#df = pd.get_dummies(df, columns=['Vehicle_Age'], drop_first=False, dummy_na=False)
df = df.drop(columns=['id', 'Region_Code'])
df

,Gender,Age,Driving_License,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,Female,45,1,0,1-2 Year,Yes,21406.0,124.0,22,0
1,Male,25,1,1,< 1 Year,No,58078.0,152.0,86,0
2,Female,57,1,1,1-2 Year,No,2630.0,26.0,214,0
3,Male,50,1,0,1-2 Year,Yes,2630.0,163.0,61,0
4,Female,21,1,0,< 1 Year,Yes,44589.0,152.0,131,0
...,...,...,...,...,...,...,...,...,...,...
346071,Female,37,1,0,1-2 Year,Yes,2630.0,156.0,218,0
346072,Male,54,1,0,> 2 Years,Yes,36962.0,122.0,36,0
346073,Female,25,1,1,< 1 Year,No,37004.0,152.0,294,0
346074,Male,22,1,1,< 1 Year,No,40338.0,152.0,67,0


In [3]:
df['Gender'] = df['Gender'].map({'Male': 1,'Female': 0})
df['Vehicle_Damage'] = df['Vehicle_Damage'].map({'Yes': 1,'No': 0})
df['Vehicle_Age'] = df['Vehicle_Age'].map({'< 1 Year': 1,'1-2 Year': 2,'> 2 Years': 3})

In [4]:
df.describe()

,Gender,Age,Driving_License,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
count,346076.000000,346076.000000,346076.000000,346076.000000,346076.000000,346076.000000,346076.000000,346076.000000,346076.000000,346076.000000
mean,0.533706,38.349943,0.997749,0.504239,1.575304,0.456865,30457.749015,114.080026,154.363307,0.033741
std,0.498863,15.732406,0.047391,0.499983,0.562429,0.498137,17063.361150,53.594294,83.653622,0.180562
min,0.000000,20.000000,0.000000,0.000000,1.000000,0.000000,2630.000000,1.000000,10.000000,0.000000
25%,0.000000,24.000000,1.000000,0.000000,1.000000,0.000000,24369.000000,55.000000,82.000000,0.000000
50%,1.000000,34.000000,1.000000,1.000000,2.000000,0.000000,31545.000000,152.000000,154.000000,0.000000
75%,1.000000,49.000000,1.000000,1.000000,2.000000,1.000000,39192.000000,152.000000,227.000000,0.000000
max,1.000000,85.000000,1.000000,1.000000,3.000000,1.000000,540165.000000,163.000000,299.000000,1.000000


In [7]:
X=df.iloc[:,:9].values
Y=df.iloc[:,9].values

In [10]:
Fav=np.sum(Y==1)/len(Y)
NonFav=np.sum(Y==0)/len(Y)
print('Favorable : {0:.2f}, Non Favorable : {1:.2f}'.format(Fav*100,NonFav*100))

Favorable : 3.37, Non Favorable : 96.63


In [11]:
run_classifieurs(X,Y,clfs)

Accuracy for RF is: 0.964 +/- 0.001 en 655.209s
Accuracy for BAG is: 0.964 +/- 0.001 en 885.794s
Accuracy for ADA is: 0.966 +/- 0.001 en 224.559s
Accuracy for ET is: 0.959 +/- 0.001 en 274.370s
Accuracy for KNN is: 0.965 +/- 0.001 en 86.364s
Accuracy for CART is: 0.936 +/- 0.001 en 14.262s
Accuracy for ID3 is: 0.937 +/- 0.002 en 16.946s
Accuracy for Stumb is: 0.966 +/- 0.001 en 1.642s
Accuracy for MLP is: 0.966 +/- 0.001 en 978.154s


In [ ]:
#Importance des variable 
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=500,random_state=1)
clf.fit(Xtrain_norm, Ytrain)
importances=clf.feature_importances_
std = np.std([tree.feature_importances_ for tree in clf.estimators_],axis=0) 
sorted_idx = np.argsort(importances)[::-1]


features =credit.columns[:-1] 
print(features[sorted_idx])
padding = np.arange(Xtrain_norm.size/len(Xtrain_norm)) + 0.5
plt.barh(padding, importances[sorted_idx],xerr=std[sorted_idx], align='center') 
plt.yticks(padding, features[sorted_idx])
plt.xlabel("Relative Importance")
plt.title("Variable Importance") 
plt.show()

In [ ]:
#Creation de pipeline
from sklearn.pipeline import Pipeline
import pickle

P = Pipeline([('SS',StandardScaler()),
                ('classifieur',MLPClassifier(activation='tanh',solver='sgd',
                    hidden_layer_sizes=(40,10),random_state=1))
                    ])
P.fit(X,Y)

pickle.dump(P,open('credit_cla.pkl','wb'))